<center><img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg"
width="1500px" height="1000px" />

<center> <font color= #004A94> <font size = 5> Microestructura y Sistemas de Trading </font> <br> <br> <font color= #047CFB> <font size = 5> Ana Laura Correa Cabrales </font> <br> <br> <font color= #047CFB> <font size = 5> Mariana López Correa </font>  <br> <br> <font color= #047CFB> <font size = 5> Jocelyn Medel Ruíz </font> <br> <br> <font color= #047CFB> 


<center><font color= #42c29b> <font size = 6> Proyecto Final </font> <br> <br> <font color= #047CFB> <font size = 5>Repositorio: <a href='https://github.com/jocelynmedel/MyST_ProyectoFinal'>Link</a></font>
    

## Fecha: 8 Diciembre de 2019

# Propósito: 
Diseñar una estrategia de trading con base a la reacción del precio ante el comunicado de indicadores macroeconómicos (Análisis fundamental).


#### Requisitos generales:

· Capital inicial: $100,000

· Riesgo por operación: $1,000


### Información sobre el indicador

#### Nonfarm payrolls (NFP)

Un pronóstico de nóminas no agrícolas es un contenido basado en sentimientos que trata de predecir cuáles serán los números de PFN y qué impacto tendrán en los mercados. En esta página, encontrará algunos artículos y herramientas que lo ayudarán a comprender cuál podría ser la perspectiva para el Informe ADP, la Tasa de desempleo, las ganancias promedio por hora, la tasa de participación en la fuerza laboral y algunos otros indicadores económicos importantes.

El ADP (Informe Nacional de Empleo) es una estimación del empleo no agrícola que se basa en una encuesta de miles de empresas del sector privado. Una de las mayores diferencias entre los datos de PFN y los datos de ADP es la muestra y la metodología del estudio. ADP Jobs Report podría mostrarnos una anticipación de las tendencias de la PFN. Algunas perspectivas se basan principalmente en el Informe de trabajos, en datos anteriores y en otros indicadores de trabajo relacionados, como las reclamaciones iniciales / continuas de desempleo.

El informe de nóminas no agrícolas mide la cantidad de empleos agregados o perdidos en la economía estadounidense durante el último mes. Se lanza generalmente el primer viernes de cada mes, a las 8:30 EST. Es publicado por el Departamento de Trabajo de los Estados Unidos.

Este informe es importante porque Estados Unidos es la economía más grande del mundo y su moneda (dólar estadounidense) es la moneda de reserva global. Las muchas economías fijan (vinculan) el valor de su moneda a la moneda de reserva, muchos productos básicos como el oro y el petróleo tienen un precio en términos de la moneda de reserva y la deuda de la economía local tiene un precio en términos de su propia moneda.

El informe de PFN, debido a su importancia para la moneda de reserva, tiende a mover todos los mercados: monedas, acciones, bonos del Tesoro, tasas de interés y también productos básicos. Lo hace inmediatamente después de la publicación de los datos económicos y, a veces, de manera tan dramática.

#### Librerias:

In [1]:
#Importar librerias
import pandas as pd
pd.options.display.max_rows = None
import math 
import pandas as pd
import numpy as np
import warnings
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.filterwarnings("ignore")
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py

#### Archivo de Precios Históricos

In [2]:
# Importación de precios históricos EUR USD
prices = pd.read_csv('precios_historicos_eurusd.csv',header=0, index_col=None, parse_dates=False)
prices['timestamp'] = [pd.to_datetime(prices.iat[i,0]).tz_localize('GMT') for i in range(len(prices))] # Timestamp a 'GMT'
prices.head()

,timestamp,open,high,low,close
0,2009-06-01 05:00:00+00:00,1.35564,1.35565,1.35383,1.35439
1,2009-06-01 05:05:00+00:00,1.35443,1.35489,1.35424,1.35474
2,2009-06-01 05:10:00+00:00,1.35480,1.35552,1.35439,1.35539
3,2009-06-01 05:15:00+00:00,1.35545,1.35549,1.35101,1.35101
4,2009-06-01 05:20:00+00:00,1.35101,1.35130,1.35021,1.35086


#### Archivo de Calendario Económico

In [3]:
# Importación de datos calendario económico
data = pd.read_csv('calendario_economico.csv',header=0, index_col=None, parse_dates=False)
data.head()

FileNotFoundError: File b'calendario_economico.csv' does not exist

In [ ]:
Nonfarm = data.loc[data["Name"] == "Nonfarm Payrolls"].reset_index(drop=True) #Resetear el indice, para encontrar calendario económico
Nonfarm.consensus.fillna(Nonfarm.previous, inplace=True) #Formar Dataframe
Nonfarm["timestamp"] = pd.to_datetime(Nonfarm["timestamp"])
Nonfarm["timestamp"] = Nonfarm["timestamp"].dt.tz_localize('GMT')
Nonfarm= Nonfarm.reset_index(drop= True)

### Escenarios del indicador

$A$ = $Actual$ ≥ $Consensus$ ≥ $Previous$

$B$ = $Actual$ ≥ $Consensus$ ≤ $Previous$

$C$ = $Actual$ ≤ $Consensus$ ≥ $Previous$

$D$ = $Actual$ ≤ $Consensus$ ≤ $Previous$

Donde:

$Actual$ : Es el valor actual del indicador.

$Consensus$ : Es el valor del indicador que se pronóstica.

$Previous$ : Es el valor anterior del indicador.

In [ ]:
#Función  para encontrar los escenarios

def clase(x):
    """Param x:DataFrame
       Return x:DataFrame con columna de clase"""
    
    x["escenario"] = x["actual"]
    for i in range(len(x["actual"])):
        if x["actual"][i] >= x["consensus"][i] >= x["previous"][i]:
            x["escenario"][i] = "A"
        elif x["actual"][i] >= x["consensus"][i] < x["previous"][i]:
            x["escenario"][i] = "B"
        elif x["actual"][i] < x["consensus"][i] >= x["previous"][i]:
            x["escenario"][i] = "C"
        elif x["actual"][i] < x["consensus"][i] < x["previous"][i]:
            x["escenario"][i] = "D"
    
    return x

In [ ]:
#Función para los periodos de 30 minutos antes y después de que se comunicó el índice 

def periodos(x,prices):
    xt = prices["timestamp"].isin(x["timestamp"])
    xt = [i for i in range(len(xt)) if xt[i] == True]
    t = [prices["timestamp"][i] for i in xt]
    xtprt = [prices.iloc[i-6:i+7] for i in xt]
    
    return xtprt,t

In [ ]:
# Función para calcular las metricas señaladas 

def metrica(x,y,z):
    dire = lambda x: x.iloc[-1,-1]-x.iloc[int(len(x)/2),1]
    alc = lambda x: x.iloc[int(len(x)/2):,1].max()-x.iloc[int(len(x)/2),1]
    baj = lambda x: x.iloc[int(len(x)/2),1] - x.iloc[int(len(x)/2):,1].min()
    vol = lambda x: x["high"].max() - x["low"].min()
    
    direccion = [dire(i.reset_index(drop=True)) for i in y]
    alcista = [alc(i.reset_index(drop=True)) for i in y]
    bajista = [baj(i.reset_index(drop=True)) for i in y]
    volatilidad = [vol(i) for i in y]
    x = x[x["timestamp"].isin(z)]
    x["direccion"] = direccion
    x["alcista"] = alcista
    x["bajista"] = bajista
    x["volatilidad"] = volatilidad
    
    return x

In [ ]:
#Aplicamos las funciones ya explicadas 

Nonfarm = clase(Nonfarm)

In [ ]:
Nonfarmper,Nonfarmt = periodos(Nonfarm,prices)

In [ ]:
Nonfarm = metrica(Nonfarm,Nonfarmper,Nonfarmt)

In [ ]:
Nonfarm.head()

In [ ]:
def escenarios(x,y,z):
    dire = lambda x: 1 if x.iloc[int(len(x)/2)]["close"] <= x.iloc[int(len(x)/2)]["open"]  else -1 
    alc = lambda x: (x.iloc[int(len(x)/2):,1].max()-x.iloc[int(len(x)/2),1])*10000
    baj = lambda x: (x.iloc[int(len(x)/2),1] - x.iloc[int(len(x)/2):,1].min())*10000
    vol = lambda x: x["high"].max() - x["low"].min()
    
    direccion = [dire(i.reset_index(drop=True)) for i in y]
    alcista = [alc(i.reset_index(drop=True)) for i in y]
    bajista = [baj(i.reset_index(drop=True)) for i in y]
    volatilidad = [vol(i) for i in y]
    
    x = x[x["timestamp"].isin(z)]
    x["direccion"] = direccion
    x["alcista"] = alcista
    x["bajista"] = bajista
    x["volatilidad"] = volatilidad
    


    return x

In [ ]:
df = escenarios(Nonfarm,Nonfarmper,Nonfarmt)
df_escenarios= df[["timestamp","escenario","direccion","alcista", "bajista", "volatilidad" ]].reset_index(drop= True)
df_escenarios.head()

In [ ]:
fig = go.Figure()
fig = make_subplots()
fig.add_trace(go.Bar(x=list(df_escenarios['escenario'].value_counts().sort_index().index), 
                     y=df_escenarios['escenario'].value_counts().sort_index(),name='Conteo' ),secondary_y=False)

fig.update_layout(title_text='Escenarios NonFarm Payrolls ')
    

fig.update_xaxes(title_text='Escenarios')
    

fig.update_yaxes(title_text=' # de Ocurrencias', secondary_y=False)
    
fig.show()

In [ ]:
A_count= list(df["escenario"]).count("A")
B_count= list(df["escenario"]).count("B")
C_count= list(df["escenario"]).count("C")
D_count= list(df["escenario"]).count("D")
print('The count of A is:', A_count)
print('The count of B is:', B_count)
print('The count of C is:', C_count)
print('The count of D is:', D_count)

In [ ]:
A = df.loc[:,"escenario"] == 'A'
df_A = df.loc[A]
df_A.head()

In [ ]:
a=list(df_A["direccion"]).count(1)
promalc_A_m= (df_A["alcista"].min())*1.15
promalc_A_M= (df_A["alcista"].max())*.85
prombaj_A_m= (df_A["bajista"].min())*1.15
prombaj_A_M= (df_A["bajista"].max())*.85

In [ ]:
B = df.loc[:,"escenario"] == 'B'
df_B = df.loc[B]
df_B.head()

In [ ]:
b=list(df_B["direccion"]).count(1)
promalc_B_m= (df_B["alcista"].min())*1.15
promalc_B_M= (df_B["alcista"].max())*.85
prombaj_B_m= (df_B["bajista"].min())*1.15
prombaj_B_M= (df_B["bajista"].max())*.85
b

In [ ]:
C = df.loc[:,"escenario"] == 'C'
df_C = df.loc[C]
df_C.head()

In [ ]:
c=list(df_C["direccion"]).count(1)
promalc_C_m= (df_C["alcista"].min())*1.15
promalc_C_M= (df_C["alcista"].max())*.85
prombaj_C_m= (df_C["bajista"].min())*1.15
prombaj_C_M= (df_C["bajista"].max())*.85


In [ ]:
D = df.loc[:,"escenario"] == 'D'
df_D = df.loc[D]
df_D.head()

In [ ]:
d=list(df_D["direccion"]).count(-1)
promalc_D_m= (df_D["alcista"].min())*1.15
promalc_D_M= (df_D["alcista"].max())*.85
prombaj_D_m= (df_D["bajista"].min())*1.15
prombaj_D_M= (df_D["bajista"].max())*.85


In [ ]:
def g_boxplot_varios(p0_data):
    """
    :param p0_data: DataFrame : Data frame con 3 columnas con datos numericos, 1 columna por boxplot
    :return:
    debugging
    p0_data = pd.DataFrame({'var1': list(np.random.normal(0,1,12)),
                            'var2': list(np.random.normal(0,1,12)),
                            'var3': list(np.random.normal(0,1,12))})
    """

    x_data = list(p0_data.columns)
    y_data = [p0_data.iloc[:, i]/max(p0_data.iloc[:, i]) for i in range(0, len(list(p0_data.columns)))]

    fig = go.Figure()

    for xd, yd in zip(x_data, y_data):
        q1 = yd.quantile(0.25)
        q3 = yd.quantile(0.75)
        iqr = q3 - q1
        out_yd = list(yd[(yd < (q1 - 1.5 * iqr)) | (yd > (q3 + 1.5 * iqr))].index)

        fig.add_trace(go.Box(y=yd, name=xd, boxpoints='all', jitter=0.5, whiskerwidth=0.5, marker_size=7,
                             line_width=1, boxmean=True, selectedpoints=out_yd))

    fig.update_layout(title='Visualizacion de todas las variables (Normalizadas)',
                      yaxis=dict(autorange=True, showgrid=True, dtick=5,
                                 gridcolor='rgb(255, 255, 255)', gridwidth=1),
                      margin=dict(l=40, r=30, b=80, t=100),
                      showlegend=False)

    fig.update_yaxes(hoverformat='.2f')

    # Mostrar figura
    # fig.show()

    return fig

In [ ]:
def operacion(x):
    """Param x:DataFrame
       Return x:DataFrame con columna de clase"""

    x["operacion"] = x["escenario"]
    x["sl"] = x["escenario"]
    x["tp"] = x["escenario"]
    x["volumen"]= x["escenario"]

    for i in range(len(x["escenario"])):
        if x["escenario"][i] == "A" :
            x["operacion"][i] = "Compra"
            x["sl"][i]=  round((prombaj_A_m + prombaj_A_M) / 2 )
            x["tp"][i]=  round((promalc_A_m + promalc_A_M) / 2 )
            x["volumen"][i]= 135500
        elif x["escenario"][i] == "B":
            x["operacion"][i] = "Compra"
            x["sl"][i]=  round((prombaj_B_m + prombaj_B_M) / 2 )
            x["tp"][i]=  round((promalc_B_m + promalc_B_M) / 2 )
            x["volumen"][i]=  232500
        elif x["escenario"][i] == "C":
            x["operacion"][i] = "Compra"
            x["sl"][i]=  round((prombaj_C_m + prombaj_C_M) / 2 )
            x["tp"][i]=  round((promalc_C_m + promalc_C_M) / 2 )
            x["volumen"][i]= 232500
        elif x["escenario"][i] == "D":
            x["operacion"][i] = "Venta"
            x["sl"][i]=  round((promalc_D_m + promalc_D_M) / 2 )
            x["tp"][i]=  round((prombaj_D_m + prombaj_D_M) / 2 )
            x["volumen"][i]= 185000
    return x

In [ ]:
df_dec= operacion(df_escenarios)

In [ ]:
df_decisiones= df_dec[["escenario", "operacion", "sl", "tp", "volumen"]].drop_duplicates().reset_index(drop=True)
df_decisiones.head()

In [ ]:
def calculo(x,y):
    
    x["resultado"]= x["volumen"]
    x["pips"]= x["volumen"]
    x["capital"]= x["volumen"] 
    
    capital= 100000 
    
    for i in range(len(x)):
        m = y[i]
       
        for j in range(len(m.iloc[6:])):
            
            if x["operacion"][i] == "Compra" :
            
                if ((x["tp"][i])/10000) + m.iloc[int(len(m)/2)]["open"] >= m.iloc[j]["high"]  or ((x["sl"][i])/10000) - m.iloc[int(len(m)/2)]["open"] <= m.iloc[j]["low"]  :
                    if ((x["tp"][i])/10000) + m.iloc[int(len(m)/2)]["open"] >=  m.iloc[j]["high"] :
                        a=   m.iloc[int(len(m)/2)]["open"]                  
                        b= ((x["tp"][i])/10000) + m.iloc[int(len(m)/2)]["open"] 
                        c= x["volumen"][i]*b - x["volumen"][i]*a
                        x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                        x["pips"][i]= (b-a)*10000
                        x["capital"][i] = capital + c 
                    else :
                        a= m.iloc[int(len(m)/2)]["open"]
                        b= m.iloc[int(len(m)/2)]["open"]- ((x["sl"][i])/10000)  
                        c= x["volumen"][i]* a - x["volumen"][i]* b 
                        x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                        x["pips"][i]= (a-b)/10000
                        x["capital"][i] = capital + c                         
                else :
                    a= m.iloc[int(len(m)/2)]["open"]   
                    b= m.iloc[int(len(m)/2)]["close"] 
                    c= x["volumen"][i]*a  - x["volumen"][i]*b
                    x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                    x["pips"][i]= (a-b)*10000
                    x["capital"][i] = capital + c       
            
            elif x["operacion"][i] == "Venta" : 
            
                if (( m.iloc[int(len(m)/2)]["open"] - x["tp"][i])/10000)  >=  m.iloc[j]["high"]  or  ((x["sl"][i])/10000) + m.iloc[int(len(m)/2)]["open"]  <= m.iloc[j]["low"] :
                    if  ((x["tp"][i])/10000) - m.iloc[int(len(m)/2)]["open"] >= m.iloc[j]["high"]  :
                        a= m.iloc[int(len(m)/2)]["open"]                  
                        b=  m.iloc[int(len(m)/2)]["open"] - ((x["tp"][i])/10000)  
                        c= x["volumen"][i]*a - x["volumen"][i]*a
                        x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                        x["pips"][i]= (b-a)*10000
                        x["capital"][i] = capital + c 
                    else :
                        a= m.iloc[int(len(m)/2)]["open"] 
                        b=((x["sl"][i])/10000) + m.iloc[int(len(m)/2)]["open"] 
                        c= x["volumen"][i]* a - x["volumen"][i]*b
                        x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                        x["pips"][i]= (a-b)*10000
                        x["capital"][i] = capital + c                         
                else :
                    
                    a= m.iloc[int(len(m)/2)]["open"]   
                    b= m.iloc[int(len(m)/2)]["close"] 
                    c= x["volumen"][i]*a - x["volumen"][i]*b
                    x["resultado"][i] = "ganadora" if c > 0  else "perdedora"
                    x["pips"][i]= (a-b)*10000
                    x["capital"][i] = capital + c       

  
    return x            

In [ ]:
df_backtest= calculo(df_dec,Nonfarmper)
df_backtest= df_backtest[["escenario", "operacion","volumen", "resultado", "pips", "capital"]]



In [ ]:
df_backtest.head()

### Evolución del Capital 

In [ ]:
fig = go.Figure()
fig = make_subplots()
fig.add_trace(go.Scatter(x=df_backtest['timestamp'], y=df_backtest['capital'],name='capital'),secondary_y=False)
# Título del grafico
fig.update_layout(title_text='Evolución del Capital')
    
# Titulo eje x
fig.update_xaxes(title_text='Fechas')
    
# Titulo eje y
fig.update_yaxes(title_text='Capital', secondary_y=False)
    
fig.show()

In [ ]:
df_box=df[[ "actual", "previous", "consensus"]]

In [ ]:
grafica = g_boxplot_varios(df_box)
py.iplot(grafica)

# Prueba

In [ ]:
prices_2 = pd.read_csv('precios_historicos_eurusd_2019.csv',header=0, index_col=None, parse_dates=False)
prices_2['timestamp'] = [pd.to_datetime(prices_2.iat[i,0]).tz_localize('GMT') for i in range(len(prices_2))] # Timestamp a 'GMT'


In [ ]:
Nonfarm_2 = data.loc[data["Name"] == "Nonfarm Payrolls"].reset_index(drop=True) #Resetear el indice, para encontrar calendario económico
Nonfarm_2.consensus.fillna(Nonfarm_2.previous, inplace=True) #Formar Dataframe
Nonfarm_2["timestamp"] = pd.to_datetime(Nonfarm_2["timestamp"])
Nonfarm_2["timestamp"] = Nonfarm_2["timestamp"].dt.tz_localize('GMT')
Nonfarm_2= Nonfarm_2.reset_index(drop= True)

In [ ]:
Nonfarm_2 = clase(Nonfarm_2)
Nonfarmper_2,Nonfarmt_2 = periodos(Nonfarm_2 ,prices_2)
Nonfarm_2 = metrica(Nonfarm_2 ,Nonfarmper_2,Nonfarmt_2)

In [ ]:
df_2 = escenarios(Nonfarm_2,Nonfarmper_2,Nonfarmt_2)
df_escenarios_2= df_2[["timestamp","escenario","direccion","alcista", "bajista", "volatilidad" ]]
df_escenarios_2 = df_escenarios_2.reset_index(drop=True)

In [ ]:
df_box_2=df_2[[ "actual", "previous", "consensus"]]

In [ ]:
grafica = g_boxplot_varios(df_box_2)
py.iplot(grafica)

In [ ]:
fig = go.Figure()
fig = make_subplots()
fig.add_trace(go.Bar(x=list(df_escenarios_2['escenario'].value_counts().sort_index().index), 
                     y=df_escenarios_2['escenario'].value_counts().sort_index(),name='Conteo' ),secondary_y=False)

fig.update_layout(title_text='Escenarios NonFarm Payrolls ')
    

fig.update_xaxes(title_text='Escenarios')
    

fig.update_yaxes(title_text=' # de Ocurrencias', secondary_y=False)
    
fig.show()

In [ ]:
df_dec_2= operacion(df_escenarios_2)
df_decisiones_2= df_dec_2[["timestamp","escenario", "operacion", "sl", "tp", "volumen"]].drop_duplicates().reset_index(drop=True)

In [ ]:
df_prueba= calculo(df_dec_2,Nonfarmper_2)

In [ ]:
df_prueba= df_prueba[["timestamp","escenario", "operacion","volumen", "resultado", "pips", "capital"]]
df_prueba

### Evolución del Capital 

In [ ]:
fig = go.Figure()
fig = make_subplots()
fig.add_trace(go.Scatter(x=df_prueba['timestamp'], y=df_prueba['capital'],name='capital'),secondary_y=False)
# Título del grafico
fig.update_layout(title_text='Evolución de Capital')
    
# Titulo eje x
fig.update_xaxes(title_text='Fechas')
    
# Titulo eje y
fig.update_yaxes(title_text='Capital', secondary_y=False)
    
fig.show()